In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()


import re
import pandas as pd
import nltk
nltk.download('words')
words = set(nltk.corpus.words.words())

In [ ]:
real_donald_trump = pd.read_csv("realdonaldtrump.csv")
trump_tweets = pd.read_csv("trumptweets.csv")

In [ ]:
cond1 = real_donald_trump['date'] >= "2017-01-20"
cond2 = real_donald_trump['date'] <= "2020-01-20"

In [ ]:
real_donald_trump = real_donald_trump.where((cond1 & cond2)).dropna()

In [ ]:
def cleaner(tweet):
    tweet = re.sub("@[A-Za-z0-9]+","",tweet) #Remove @ sign
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
    tweet = " ".join(tweet.split())
    tweet = tweet.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
    tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet)
         if w.lower() in words or not w.isalpha())
    return tweet
    

real_donald_trump['content_clean'] = real_donald_trump['content'].apply(cleaner)

In [ ]:
word_dict = {'manipulate':-1,'manipulative':-1,'jamescharlesiscancelled':-1,'jamescharlesisoverparty':-1,
            'pedophile':-1,'pedo':-1,'cancel':-1,'cancelled':-1,'cancel culture':0.4,'teamtati':-1,'teamjames':1,
            'teamjamescharles':1,'liar':-1}


sid = SentimentIntensityAnalyzer()
sid.lexicon.update(word_dict)

list1 = []
for i in real_donald_trump['content_clean']:
    list1.append((sid.polarity_scores(str(i)))['compound'])

In [ ]:
real_donald_trump['sentiment'] = list1

def sentiment_category(sentiment):
    label = ''
    if(sentiment > 0.5):
        label = 'positive'
    else:
        label = 'negative'
    return(label)

real_donald_trump['sentiment_category'] = real_donald_trump['sentiment'].apply(sentiment_category)

In [ ]:
real_donald_trump

In [ ]:
neg = real_donald_trump[real_donald_trump['sentiment_category']=='negative']
neg = neg.groupby(['date'],as_index=False).count()

pos = real_donald_trump[real_donald_trump['sentiment_category']=='positive']
pos = pos.groupby(['date'],as_index=False).count()

pos = pos[['date','id']]
neg = neg[['date','id']]

In [ ]:
neg_date = neg['date'].tolist()

In [ ]:
real_donald_trump['content'].loc[real_donald_trump['date'] == neg_date[50]].tolist()